<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/BiDAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk import wordnet
from nltk.tokenize import word_tokenize
import string

import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter
nlp = spacy.load('en')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Build a list of dictionaries with ( context , question , label ) as keys**

In [2]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list

# **Function to pre-process paragraph of text**

In [3]:
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', '', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        #Replacing special worsds
        #document = re.sub(r'නැකඩ්', " ", document)
        #document = re.sub(r'අඩීචි'," ", document)
        #document = re.sub(r'සොංටැං'," ",document)
        #document = re.sub(r'``', " ", document)
        #document = re.sub(r'[^\w\s]','',document)
        #document = re.sub(r'\_','',document)
        # Converting to Lowercase
        document = document.lower()

        #removing digits from the corpus since FastText unable to embedd digits
        #document= re.sub(r'\b\d+\b', '', document)
        

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

# **Function to create list of Context, Question and Context+Question lists**

In [4]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['text_translated']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  table1 = str.maketrans('', '', string.punctuation)
  stripped_context = [w.translate(table1) for w in context]

  table2 = str.maketrans('', '', string.punctuation)
  stripped_question = [w.translate(table2) for w in question]

  table3 = str.maketrans('', '', string.punctuation)
  stripped_answer = [w.translate(table2) for w in answer]

  lis=stripped_context+stripped_question
  return lis,stripped_context,stripped_question,stripped_answer

# **Function to build vocabulary of words , word to index and index to word mapping**

In [5]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=word_tokenize(sentences)
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=word_tokenize(line)
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab


# **Function to create character vocabulary and character to index mapping**

In [6]:
def build_char_vocab(vocab_text):

    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'<unk>')
    char_vocab.insert(1,'<pad>')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

# **Function to create id's for pre-processed context words**

In [7]:
def context_to_ids(text, word2idx):
   

    context_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

# **Function to create id's for pre-processed questions**

In [8]:
def question_to_ids(text, word2idx):
 

    question_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [ ]:
from google.colab import files
uploaded = files.upload()